# Disjunctive Programming

In [ ]:
from ortools.linear_solver import pywraplp
import pandas as pd
import random

random.seed(42)  # 保证可重复性


nutrients = ["Vit A", "Vit B", "Vit C", "Vit D", "Minerals", "Fiber", "Protein"]


comp_data = {}
components = [f"Comp_{i}" for i in range(1, 11)]

for comp in components:
    comp_data[comp] = {
        "cost": round(random.uniform(5, 25), 2),  # 成本在5-25之间
    }
    for nutrient in nutrients:
        comp_data[comp][nutrient] = round(random.uniform(0.1, 0.9), 3)

# 更复杂的产品需求
prod_req = {
    "Vit A": {"lb": 0.1, "ub": 0.4},
    "Vit B": {"lb": 0.2, "ub": 0.6},
    "Vit C": {"lb": 0.05, "ub": 0.3},
    "Vit D": {"lb": 0.1, "ub": 0.5},
    "Minerals": {"lb": 0.15, "ub": 0.4},
    "Fiber": {"lb": 0.1, "ub": 0.8},
    "Protein": {"lb": 0.2, "ub": 0.7},
}

# 更多的互斥对组合
excl_pairs = [
    ("Comp_1", "Comp_2"),  # 成分1和2互斥
    ("Comp_3", "Comp_4"),  # 成分3和4互斥  
    ("Comp_5", "Comp_6"),  # 成分5和6互斥
    ("Comp_7", "Comp_8"),  # 成分7和8互斥
]

# 添加一些三元互斥组（三个成分中最多只能选一个）
excl_triples = [
    ("Comp_1", "Comp_3", "Comp_5"),
    ("Comp_2", "Comp_4", "Comp_6"),
]


solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    raise Exception("无法创建求解器")

# Decision variables
x = {}
for comp in comp_data.keys():
    x[comp] = solver.NumVar(0, solver.infinity(), f'x_{comp}')

# Bool variables
y = {}
for pair in excl_pairs:
    y[pair] = solver.BoolVar(f'y_{pair}')

# Min total cost.
cost_expr = sum(x[c] * comp_data[c]["cost"] for c in comp_data.keys())
solver.Minimize(cost_expr)

# blend proportion constraint
solver.Add(sum(x[c] for c in comp_data.keys()) == 1, 'mass_fraction')

# 成分约束
for req, bounds in prod_req.items():
    # 下限约束
    lb_expr = sum(x[c] * comp_data[c][req] for c in comp_data.keys())
    solver.Add(lb_expr >= bounds["lb"], f'lb_{req}')
    
    # 上限约束
    ub_expr = sum(x[c] * comp_data[c][req] for c in comp_data.keys())
    solver.Add(ub_expr <= bounds["ub"], f'ub_{req}')

# Here is disjunctive constraints!
M = 100  # 大M值
for pair in excl_pairs:
    a, b = pair
    # 如果y=1，则x[a] <= M*1, x[b] <= 0 (即x[b]=0)
    # 如果y=0，则x[a] <= 0, x[b] <= M*1
    solver.Add(x[a] <= M * y[pair], f'excl_{a}_active')
    solver.Add(x[b] <= M * (1 - y[pair]), f'excl_{b}_active')

# 求解问题
status = solver.Solve()

# 输出结果
if status == pywraplp.Solver.OPTIMAL:
    print("Found Optimal Sol!")
    print(f"Min cost: {solver.Objective().Value()}")
    print("\nProportion:")
    for comp in comp_data.keys():
        if x[comp].solution_value() > 1e-6:  # 只显示非零值
            print(f"{comp} = {x[comp].solution_value():.6f}")
    
    print("\nExclusive pairs:")
    for pair in excl_pairs:
        print(f"y{pair} = {y[pair].solution_value()}")
        
    # 验证约束
    print("\n约束验证:")
    total = sum(x[c].solution_value() for c in comp_data.keys())
    print(f"质量分数总和: {total:.6f}")
    
    for req in prod_req.keys():
        content = sum(x[c].solution_value() * comp_data[c][req] for c in comp_data.keys())
        print(f"{req}含量: {content:.6f} (要求: [{prod_req[req]['lb']}, {prod_req[req]['ub']}])")
else:
    print("未找到最优解")
    if status == pywraplp.Solver.INFEASIBLE:
        print("问题不可行")
    elif status == pywraplp.Solver.UNBOUNDED:
        print("问题无界")

# 可选：显示更详细的求解信息
print(f"\n求解状态: {status}")
print(f"求解时间: {solver.wall_time()} ms")
print(f"迭代次数: {solver.iterations()}")

Found Optimal Sol!
Min cost: 13.778840895903448

Proportion:
Comp_2 = 0.478403
Comp_3 = 0.325543
Comp_9 = 0.196054

Exclusive pairs:
y('Comp_1', 'Comp_2') = 0.0
y('Comp_3', 'Comp_4') = 1.0
y('Comp_5', 'Comp_6') = 0.0
y('Comp_7', 'Comp_8') = 0.0

约束验证:
质量分数总和: 1.000000
Vit A含量: 0.386562 (要求: [0.1, 0.4])
Vit B含量: 0.430550 (要求: [0.2, 0.6])
Vit C含量: 0.300000 (要求: [0.05, 0.3])
Vit D含量: 0.369427 (要求: [0.1, 0.5])
Minerals含量: 0.400000 (要求: [0.15, 0.4])
Fiber含量: 0.470450 (要求: [0.1, 0.8])
Protein含量: 0.496776 (要求: [0.2, 0.7])

求解状态: 0
求解时间: 8 ms
迭代次数: 16
